In [5]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt
from tqdm import tqdm

In [6]:
data_path = "../data/"

documents = [line.strip() for line in open(data_path + "2016-10-20.txt", "r").readlines()]

print(documents[:3])

['', '19  1990  52 1 22', '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 

In [7]:
preprocessed_documents = []

for line in tqdm(documents):
    # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
    if line and not line.replace(' ', '').isdecimal():
        preprocessed_documents.append(line)
        
print(preprocessed_documents[:3])

100%|██████████| 30091/30091 [00:00<00:00, 179617.57it/s]

['오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33

# Tokenizing

In [8]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [11]:
tokenizer = CustomTokenizer(Okt())

vectorizer = CountVectorizer(tokenizer=tokenizer, max_features=3000)
embeddings = vectorizer.fit_transform(preprocessed_documents)
print(embeddings.shape)

/Users/godpeny/Code/venv/laboratory/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(27540, 3000)


In [17]:
vocab = vectorizer.get_feature_names_out()
print(vocab)

id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}
print(id2token)

['00' '000' '000원' ... '힐링' '힘든' '힙합']
{0: '00', 1: '000', 2: '000원', 3: '01', 4: '02', 5: '10', 6: '100', 7: '1000', 8: '100만', 9: '101', 10: '10년', 11: '10분', 12: '10시', 13: '10월', 14: '10일', 15: '11', 16: '11시', 17: '11월', 18: '11일', 19: '12', 20: '12월', 21: '12일', 22: '13', 23: '13일', 24: '14', 25: '14일', 26: '15', 27: '15일', 28: '16', 29: '16일', 30: '17', 31: '17일', 32: '18', 33: '18일', 34: '19', 35: '19일', 36: '1년', 37: '1만', 38: '1시', 39: '1억', 40: '1월', 41: '1일', 42: '1조', 43: '1천', 44: '1회', 45: '20', 46: '200', 47: '2000', 48: '2005년', 49: '2007년', 50: '2008년', 51: '2009년', 52: '2010년', 53: '2011년', 54: '2012년', 55: '2013년', 56: '2014년', 57: '2015년', 58: '2016', 59: '2016년', 60: '2017', 61: '2017년', 62: '2018년', 63: '2020년', 64: '20일', 65: '21', 66: '21일', 67: '22', 68: '22일', 69: '23', 70: '23일', 71: '24', 72: '24일', 73: '25', 74: '25일', 75: '26', 76: '26일', 77: '27', 78: '27일', 79: '28', 80: '28일', 81: '29', 82: '29일', 83: '2년', 84: '2만', 85: '2분', 86: '2시', 87: '2월', 88: '

# Modeling

In [15]:
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents,"sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

Batches:   0%|          | 0/138 [00:00<?, ?it/s]

In [18]:
qt = TopicModelDataPreparation()

training_dataset = qt.load(train_contextualized_embeddings, embeddings, id2token)

ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=5)
ctm.fit(training_dataset)

Epoch: [5/5]	 Seen Samples: [137700/137700]	Train Loss: 895.5107978593636	Time: 0:01:06.032183: : 5it [05:24, 65.00s/it]


In [19]:
ctm.get_topics(5)

defaultdict(list,
            {0: ['토론', '시간', '미국', '라스베이거스', '트럼프'],
             1: ['경찰', '강북', '오패산', '했습니다', '터널'],
             2: ['뉴시스', '공감', '영상', '언론', '사진'],
             3: ['당장', '이란', '검증', '한계', '어렵다'],
             4: ['이탈', '동아닷컴', '포토', '일간스포츠', '을지로'],
             5: ['오는', '월화드라마', '아모', '사는', '남자'],
             6: ['혐의', '밝혔다', '발생', '조사', '경찰'],
             7: ['방송', '모습', '출연', '라며', '화신'],
             8: ['으로', '했다', '에서', '하고', '있다'],
             9: ['분양', '이상', '금리', '단지', '대출'],
             10: ['배우', '라며', '라고', '모습', '방송'],
             11: ['에서', '했다', '하는', '하고', '사람'],
             12: ['으로', '장관', '국방', '한국', '위협'],
             13: ['검증', '기본', '당장', '형성', '어렵다'],
             14: ['사업', '노조', '위해', '파업', '20일'],
             15: ['최근', '증권', '보다', '000', '000원'],
             16: ['했다', '에서', '있다', '으로', '있는'],
             17: ['공감', '언론', '뉴시스', '영상', '7470'],
             18: ['참석', '배우', '오는', '제작발표회', '아모'],
             19: ['어렵다', '이란',

In [20]:
ctm.get_topic_lists(5)

[['토론', '시간', '미국', '라스베이거스', '트럼프'],
 ['경찰', '강북', '오패산', '했습니다', '터널'],
 ['뉴시스', '공감', '영상', '언론', '사진'],
 ['당장', '이란', '검증', '한계', '어렵다'],
 ['이탈', '동아닷컴', '포토', '일간스포츠', '을지로'],
 ['오는', '월화드라마', '아모', '사는', '남자'],
 ['혐의', '밝혔다', '발생', '조사', '경찰'],
 ['방송', '모습', '출연', '라며', '화신'],
 ['으로', '했다', '에서', '하고', '있다'],
 ['분양', '이상', '금리', '단지', '대출'],
 ['배우', '라며', '라고', '모습', '방송'],
 ['에서', '했다', '하는', '하고', '사람'],
 ['으로', '장관', '국방', '한국', '위협'],
 ['검증', '기본', '당장', '형성', '어렵다'],
 ['사업', '노조', '위해', '파업', '20일'],
 ['최근', '증권', '보다', '000', '000원'],
 ['했다', '에서', '있다', '으로', '있는'],
 ['공감', '언론', '뉴시스', '영상', '7470'],
 ['참석', '배우', '오는', '제작발표회', '아모'],
 ['어렵다', '이란', '당장', '변경', '가까운'],
 ['있는', '하는', '있다', '에서', '한다'],
 ['포토', '취하', '갖고', '중구', '을지로'],
 ['했습니다', '회의', '국회', '밝혔습니다', '북한'],
 ['방송', '에게', '사람', '라고', '라며'],
 ['함께', '있는', '부터', '배우', '하며'],
 ['에서', '했다', '하고', '회의', '대한'],
 ['이번', '행사', '부터', '통해', '제공'],
 ['사랑', '배우', '남자', '사는', '제작발표회'],
 ['서울', '디자이너', '패션위크', '플라자', '20